# Load List of Kings and Queens
This Scripts generates a list with all members of the Kantonsrat back to the age of the dinosaurs. It stores as well who was when president of the Kantonsrat.  

Load from xlsx because:
* CSVs are corrupt
* API from Parlamentsdienste is a pain  

Download at the bottom: https://www.zh.ch/de/politik-staat/wahlen-abstimmungen/kantons-regierungsratswahlen/mitglieder-kantonsrats-ab-1803.html#-373901206

In [1]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import numpy as np
import json

In [2]:
df_person = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Personen')
df_funktion = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Funktionen')
df_partei = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Parteien')
df_einsitz = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Einsitze')

In [3]:
df_person = df_person[df_person.DATUM_GEBURT_JAHR.notna()]
df_person = df_person[df_person.DATUM_GEBURT_JAHR> 1900]
df_einsitz = df_einsitz[df_einsitz.DATUM_EINTRITT_JAHR.notna()]
df_einsitz = df_einsitz[df_einsitz.RAT == 'Kantonsrat']
df_funktion = df_funktion[df_funktion.DATUM_VON_JAHR.notna()]
df_funktion = df_funktion[df_funktion.FUNKTIONSBEZEICHNUNG.notna()]
df_partei = df_partei[df_partei.PARTEIBEZEICHNUNG.notna()]

records = []

for i, row in df_person.iterrows():

    record = {
        'id': row['ID_PERSON_NEW'],
        'name': row['NACHNAME'].strip(),
        'vorname': row['VORNAME'].strip(),
        'geschlecht': row['GESCHLECHT'].strip(),
        'jahrgang': row['DATUM_GEBURT_JAHR'],
        'einsitz': [],
        'partei': [],
        'funktion': []
    }

    # Add Einsitz
    df = df_einsitz[df_einsitz.ID_PERSON_NEW == row['ID_PERSON_NEW']]
    for y, row2 in df.iterrows():

        if np.isnan(row2['DATUM_AUSTRITT_JAHR']):
            end = datetime(2100, 12, 31)
        else:
            end = datetime(int(row2['DATUM_AUSTRITT_JAHR']), int(row2['DATUM_AUSTRITT_MONAT']), int(row2['DATUM_AUSTRITT_TAG']))

        record['einsitz'].append({
            'start': datetime(int(row2['DATUM_EINTRITT_JAHR']), int(row2['DATUM_EINTRITT_MONAT']), int(row2['DATUM_EINTRITT_TAG'])),
            'end': end
        })

    # Add Party
    df = df_partei[df_partei.ID_PERSON_NEW == row['ID_PERSON_NEW']]
    try:
        for y, row2 in df.iterrows():
            if not(np.isnan(row2['DATUM_BIS_MONAT'])) and not(np.isnan(row2['DATUM_BIS_JAHR'])):
                end = datetime(int(row2['DATUM_BIS_JAHR']), int(row2['DATUM_BIS_MONAT']), int(row2['DATUM_BIS_TAG']))
            else:
                end = datetime(2100, 12, 31)

            if np.isnan(row2['DATUM_VON_JAHR']):
                start = record['einsitz'][0]['start']
            else:
                start = datetime(int(row2['DATUM_VON_JAHR']), int(row2['DATUM_VON_MONAT']), int(row2['DATUM_VON_TAG']))

            record['partei'].append({
                'bezeichnung': row2['PARTEIBEZEICHNUNG'],
                'start': start,
                'end': end
            })
        # Reorder
        record['partei'] = sorted(record['partei'], key = lambda x: x['start'])
    except:
        print(row2)
        raise
        
    # Add Funktion
    df = df_funktion[df_funktion.ID_PERSON_NEW == row['ID_PERSON_NEW']]
    try:
        for y, row2 in df.iterrows():

            # Skip if "Mitglied"
            if row2['FUNKTIONSBEZEICHNUNG'].lower() == 'mitglied': continue

            # Assume, that Amtsjahr starts at 1. Mai and ends at 30. April
            if not np.isnan(row2['DATUM_VON_JAHR']):
                day = 1 if np.isnan(row2['DATUM_VON_TAG']) else int(row2['DATUM_VON_TAG'])
                month = 5 if np.isnan(row2['DATUM_VON_MONAT']) else int(row2['DATUM_VON_MONAT'])
                start = datetime(int(row2['DATUM_VON_JAHR']), month, day)
            else:
                start = datetime(2100, 12, 31)
                print("Hmmm")


            if np.isnan(row2['DATUM_BIS_JAHR']):
                end = datetime(2100, 12, 31)
            else:
                day = 30 if np.isnan(row2['DATUM_BIS_TAG']) else int(row2['DATUM_BIS_TAG'])
                month = 4 if np.isnan(row2['DATUM_BIS_MONAT']) else int(row2['DATUM_BIS_MONAT'])
                end = datetime(int(row2['DATUM_BIS_JAHR']), month, day)                

            record['funktion'].append({
                'bezeichnung': row2['FUNKTIONSBEZEICHNUNG'],
                'start': start,
                'end': end
            })

    except:
        print(row2)
        raise
    records.append(record)

In [4]:
# Drop by ID (duplicated entries)

# Not needed in new file version
"""
print(len(records))
remove = None
for r in records:
    if r['id'] == 21252:
        remove = r
        break

if remove:
    records.remove(remove)

print(len(records))
"""

"\nprint(len(records))\nremove = None\nfor r in records:\n    if r['id'] == 21252:\n        remove = r\n        break\n\nif remove:\n    records.remove(remove)\n\nprint(len(records))\n"

In [5]:
# Fix Christoph Fischbach
r = list(filter(lambda x: x['id'] == 21362, records))[0]
r['einsitz'].append({
    "start": "2021-11-08 00:00:00",
    "end": "2100-12-31 00:00:00"
})

r['partei'].append({
    "bezeichnung": "SP",
    "start": "2021-11-08 00:00:00",
    "end": "2100-12-31 00:00:00"
})

In [6]:
# Fix Cristina Cortellini
r = list(filter(lambda x: x['id'] == 21445, records))[0]
r['name'] = 'Wyss-Cortellini'

In [7]:
# Add manually
# Not needed anymore
"""
records.append({
    "id": 0,
    "name": "Hoss-Blatter",
    "vorname": "Corinne",
    "geschlecht": "w",
    "jahrgang": 1937.0,
    "einsitz": [{"start": "2020-02-24 00:00:00", "end": "2100-12-31 00:00:00"}],
    "partei": [{"bezeichnung": "FDP", "start": "2020-02-24 00:00:00", "end": "2100-12-31 00:00:00"}],
    "funktion": []})
"""

'\nrecords.append({\n    "id": 0,\n    "name": "Hoss-Blatter",\n    "vorname": "Corinne",\n    "geschlecht": "w",\n    "jahrgang": 1937.0,\n    "einsitz": [{"start": "2020-02-24 00:00:00", "end": "2100-12-31 00:00:00"}],\n    "partei": [{"bezeichnung": "FDP", "start": "2020-02-24 00:00:00", "end": "2100-12-31 00:00:00"}],\n    "funktion": []})\n'

In [8]:
with open(Path('../export/mitglieder.json'), 'w', encoding='utf-8') as f:
    json.dump(records, f, default=str, ensure_ascii=False, indent=2)